In [17]:
from dotenv import load_dotenv
env = load_dotenv()

In [8]:
from langgraph.graph import StateGraph, END
from langchain_core.messages import AnyMessage, SystemMessage, HumanMessage, ToolMessage
from langchain_core.output_parsers import JsonOutputParser
from langchain_experimental.llms.ollama_functions import OllamaFunctions
from IPython.display import Image
import IPython

from typing import TypedDict, Annotated, Dict, Any
import operator
import json

In [7]:
class AgentState(TypedDict):
    restaurant: str
    output: str

In [31]:
from langchain.pydantic_v1 import BaseModel, Field
from langchain.tools import tool
from langchain_community.tools.tavily_search import TavilySearchResults
from langchain_community.tools import DuckDuckGoSearchRun
import requests

class SearchInput(BaseModel):
    restaurant: str = Field(description="the restaurant name and its location")

search_tool = DuckDuckGoSearchRun()

In [36]:
search_tool.invoke("Chick Fil A menu")

"Chick-fil-A Menu and Prices. 3.9 based on 1,673 votes Chick-fil-A Coupons Chick-fil-A Nutrition Facts. Choose My State. AL AZ AR CA CO DE FL GA ID IL IN IA KS KY LA MD MA MI MN MS MO NE NH NJ NM NY NC OH OK PA SC TN TX UT VA DC WV WI WY All Less. Chick-fil-A Menu / Chick-fil-A Catering Menu. Last Update: 2024-04-16. Order Online. Breakfast ... Chick-fil-A, Inc. is the third largest quick-service restaurant company in the United States, known for its freshly-prepared food, signature hospitality and unique franchise model. More than 200,000 Team Members are employed by independent owner-operators in more than 3,000 restaurants across the United States, Canada, and Puerto Rico. Atlanta-based Chick-fil-A, Inc. is a restaurant company known for the Original Chick-fil-A ® Chicken Sandwich and signature hospitality. Represented by more than 190,000 Team Members, Operators and Staff, Chick-fil-A ® restaurants serve guests freshly prepared food at more than 2,800 restaurants in 48 states, Wash

In [27]:
tools = [search_tool]

In [ ]:
class Model:
    # Define the model
    def __init__(self, model, tools, system=""):
        self.system = system
        graph = StateGraph(AgentState)
        graph.add_node("llm", self.call_openai)
        graph.add_node("action", self.take_action)
        graph.add_node("output-parser", self.parse_output_to_string)
        graph.add_conditional_edges(
            "llm",
            self.exists_action,
            {True: "action", False: "output-parser"}
        )
        graph.add_edge("action", "llm")
        graph.add_edge("output-parser", END)
        graph.set_entry_point("llm")
        self.graph = graph.compile()
        self.tools = {t.name: t for t in tools}
        self.model = model.bind_tools(tools)

    # Check if llm requires action 
    def exists_action(self, state: AgentState):
        result = state['messages'][-1]
        return len(result.tool_calls) > 0
    # Run a tool ordered by the model
    def take_action(self, state: AgentState):
        tool_calls = state['messages'][-1].tool_calls
        results = []
        for t in tool_calls:
            print(f"Calling: {t}")
            result = self.tools[t['name']].invoke(t['args'])
            results.append(ToolMessage(tool_call_id=t['id'], name=t['name'], content=str(result)))
        print("Back to the model!")
        return {'messages': results}
    # Perform inference on the gathered context
    def classify(self, state: AgentState):
        messages = state['messages']
        if self.system:
            messages = [SystemMessage(content=self.system)] + messages
        message = self.model.invoke(messages)
        return {'messages': [message]}

    # def parse_output_to_string(self, state: AgentState):
    #     message = state['messages'][-1]
    #     json_output = JsonOutputParser(pydantic_object=Restaurants).invoke(message)
    #     return {'json_output': json_output}

In [ ]:
from langchain_core.prompts import PromptTemplate

prompt = PromptTemplate.from_template(
"""You are an AI tasked with classifying compatible diet types from a menu.

----------
{ menu }
----------
You output MUST be structured list of one or more of the provided diet types,
{ diet_types }
----------
DO NOT respond as an AI assistant, only return a list of compatible diet types. 
You may return no diet types if no fit, or one diet type if only one fits.

EXAMPLE:
["diet_type_1", "diet_type_2", ...]
"""
)

prompt.format(context=context)

In [ ]:
llm = OllamaFunctions(
    model="phi3", 
    keep_alive=-1,
    format="json"
    )
)

model = Model(llm, tools, system=prompt)

In [ ]:
Image(model.graph.get_graph().draw_png())

In [ ]:
input = [HumanMessage(content="CRISP Rochester")]
result = model.graph.invoke({"restaurant": input})